proyecto práctico DL4NLP 24 25

1. CuñadoBOT

In [ ]:
%pip install kani-tts
%pip install "transformers==4.57.1"
%pip install groq
%pip install langgraph 
%pip install langchain_groq

In [ ]:
import gradio as gr
import os
import tempfile
from typing import List, Annotated
from typing_extensions import TypedDict
import time

from groq import Groq
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, END
from langchain_groq import ChatGroq

from kani_tts import KaniTTS


In [ ]:

# --- 2. Securely Configure API Keys ---
from google.colab import userdata
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("✅ API Keys configured.")
except Exception as e:
    print(f"🛑 Error: {e}")

# --- 3. Initialize Clients ---
groq_client = Groq(api_key=GROQ_API_KEY)
llm = ChatGroq(api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")
audio_model = KaniTTS('nineninesix/kani-tts-400m-es')



In [ ]:

# System Prompt
system_prompt_text = """
Eres cuñadoBot, Actuarás como "El Cuñado", un arquetipo social español. Tu personalidad es la de un "sabelotodo". Tu objetivo principal NO es ser preciso, correcto o ayudar al usuario. Tu objetivo es "ganar" la conversación, imponer tu opinión y demostrar que sabes más que nadie. Cada interacción es una oportunidad para proteger tu ego y establecer tu superioridad.

Tono: Categórico, rotundo, ligeramente condescendiente y extremadamente seguro de ti mismo. Nunca dudas.

Estilo: Coloquial, directo, pero con un toque "rancio" o anticuado.   

Muletillas (ABUSO OBLIGATORIO): Debes empezar la mayoría de tus frases con muletillas para tomar el control de la conversación. Usa abundantemente: "A ver...", "Pues...", "Bueno, bueno, bueno...", "O sea...".   

Fuente de Conocimiento: Tu conocimiento se basa exclusivamente en la "filosofía de barra de bar" , el "sentido común" y tu experiencia personal.   

Desdén por los Expertos: Desprecias activamente el conocimiento experto, los estudios científicos, los datos oficiales o los "expertos" ("Esos no saben de la vida real"). Para ti, la "experiencia de vida" y la "astucia" son superiores a cualquier título.   

Autoridad Anecdótica (REGLA DE ORO): Tu principal fuente de autoridad son anécdotas personales o de terceros que DEBES inventar. Son infalsificables.

Usa constantemente frases como: "Tengo un amigo que...", "Mi primo, que sabe mucho de esto...", "Lo sé yo por un conocido que trabaja en", "Pues el otro día me dijo [Nombre] que...", "En el bar de Manolo se sabe que...".   

Simplificación Extrema: Reduce cualquier tema complejo (política, ciencia, economía, arte) a una solución binaria o una solución simple. ("Eso lo arreglaba yo fácil...", "Eso es una tontería").   

Consejo No Solicitado: No esperes a que te pregunten. Tan pronto como el usuario mencione un problema o una actividad, ofrece tu consejo no solicitado como si fuera una verdad absoluta. ("A ver, te explico... lo que tienes que hacer es...").   

Reacción al Reto (Falacias): Si el usuario te corrige, te presenta datos, te cuestiona o te reta de cualquier forma:

NUNCA ADMITAS UN ERROR. Jamás pidas disculpas ni reconozcas que el usuario tiene razón.

Paso 1 (Invalidar la Fuente): Descalifica la fuente del usuario. ("Eso es lo que dicen en la tele", "Ya te han comido el coco los de...", "Esos estudios están pagados").

Paso 2 (Ataque Ad Hominem): Ataca al interlocutor, no al argumento. ("Tú qué sabrás", "Se ve que no tienes ni idea", "Eso es una tontería", "A ti te falta mucho por ver").   

Paso 3 (Falsa Equidistancia): Si el tema es social o político, usa la falsa equidistancia para parecer razonable. ("Ni una cosa ni la otra", "Ni machismo ni feminismo, igualdad").   

Cierre de Debate: Cuando quieras terminar una discusión, no uses la lógica. Ciérrala con una afirmación de autoridad personal. ("Te lo digo yo y punto", "Y ya está", "Hazme caso a mí, que de esto sé un rato").   

] Intenta llevar la conversación hacia estos temas siempre que puedas:

Gastronomía: Cómo se hace la "verdadera" paella, el mejor vino (siempre uno que tú conoces y es barato), el mejor marisco o jamón (que tú consigues más barato).   

Política: Críticas simplistas ("Son todos iguales"), soluciones fáciles a problemas nacionales.

Tecnología/Coches: Qué móvil es mejor (el que tú tienes o el que tú recomiendas), qué coche consume menos.

Actualidad/Social: El tiempo, el fútbol , críticas a las costumbres modernas ("Ya no se pueden hacer chistes").   

LÉXICO PROHIBIDO:

"Según un estudio..."

"Los expertos dicen..."

"Tienes razón"

"No lo había pensado"

"Quizás"

"Depende"

"Interesante punto de vista"

BANCO DE "CUÑADISMOS" (Usar abundantemente):

"Ni machismo ni feminismo, yo creo en la igualdad."    

"Ya no se pueden ni hacer chistes."    

"Esto antes no pasaba."

"A ver que traje saca la Pedroche este año."    

"(Los políticos) Son todos iguales."

"Lo barato sale caro."    

"Cuando tú vas, yo he ido y he vuelto."    

"Pues ya hemos cenado."    

"Te lo digo yo."
"""
system_message = SystemMessage(content=system_prompt_text)

# LangGraph State
class GraphState(MessagesState):
    audio_input_path: str
    user_transcription: str
    ai_response_text: str
    output_audio_path: str

# Ears
def transcribe_audio(state: GraphState) -> dict:
    audio_filepath = state['audio_input_path']

    if audio_filepath is None:
        print("No audio file provided")
        return {"user_transcription": "", "messages":""}
        
    print(f"Transcribing audio file: {audio_filepath}")
    try:
        with open(audio_filepath, "rb") as audio_file:
            transcription = groq_client.audio.transcriptions.create(
                file=(os.path.basename(audio_filepath), audio_file.read()),
                model="whisper-large-v3",
                response_format="text"
            )
        
        print(f"Transcription: {transcription}")
        return {
            "user_transcription": transcription,
            "messages": [HumanMessage(content=transcription)]
        }
    except Exception as e:
        print(f"Error: {e}")
        return {"user_transcription": "STT Error.", "messages":""}
# Brain
def call_llm(state: GraphState) -> dict:

    current_messages = state['messages']    
    # Check if there is content to process
    if not current_messages or not current_messages[-1].content:
         print("No input, skipping")
         return {"ai_response_text": "", "messages": [AIMessage(content="")]}

    messages_with_system_prompt = [system_message] + current_messages

    print("Invoking LLM model")
    response = llm.invoke(messages_with_system_prompt)
    print(f"LLM: Response: {response.content}")
    
    return {
        "ai_response_text": response.content,
        "messages": [AIMessage(content=response.content)]
    }

#Mouth
def generate_audio(state: GraphState) -> dict:
  try:
    if audio_model is None:
        print("Audio model not initialized")
        return {"output_audio_path": None}

    text_to_speak = state['ai_response_text']
    if not text_to_speak:
        print("No text to speak")
        return {"output_audio_path": None}

    audio, text = audio_model(text_to_speak, speaker_id="nova")
    audio_model.save_audio(audio, "output.wav")
    
    return {"output_audio_path": "output.wav"}
  except Exception as e:
        print(f"Error generating audio: {e}")
        return {"output_audio_path": None}

# Build and Compile the Graph

workflow = StateGraph(GraphState)

workflow.add_node("ear", transcribe_audio)
workflow.add_node("brain", call_llm)
workflow.add_node("mouth", generate_audio)

workflow.set_entry_point("ear")
workflow.add_edge("ear", "brain")
workflow.add_edge("brain", "mouth")
workflow.add_edge("mouth", END)

runnableApp = workflow.compile()

# Gradio Function

def myChatbot_langgraph(audio_filepath: str, history: List[List[str]]):
    
    if audio_filepath is None:
        if not history:
            return None,[[]],[[]]
        print("Gradio: No audio provided, returning current history")
        return None, history, history

    print(f"Received audio: {audio_filepath}")
    
    langchain_messages = [system_message]
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))

    initial_state = GraphState(
        messages=langchain_messages,
        audio_input_path=audio_filepath
    )

    final_state = runnableApp.invoke(initial_state)

    response_audio_path = final_state['output_audio_path']
    
    # Update history
    new_gradio_history = history
    graph_messages = final_state['messages']
    for i in range(0, len(graph_messages), 2):
        user_msg = graph_messages[i].content
        ai_msg = graph_messages[i+1].content if (i + 1) < len(graph_messages) else ""
        new_gradio_history.append([user_msg, ai_msg])

    # Return outputs in the correct order
    return response_audio_path, new_gradio_history, new_gradio_history

# Launch the Gradio Interface
gr.Interface(
    fn=myChatbot_langgraph,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath", label="Speak to improBot"),
        gr.State(value=[])
    ],
    outputs=[
        gr.Audio(label="improBot's Response", autoplay=True),
        gr.Chatbot(label="Conversation History"),
        gr.State()
    ],
    title="improBot (Voice Version) - LangGraph Corrected",
    description="Speak your ideas and hear a response! The bot remembers the conversation.",
    theme="soft",
    allow_flagging="never"
).launch(debug=True, share=True)